# From the previously generated points.geojson, and the event-based jsons that have each point's WSE data, we can create a json for each point across events.
## event-based jsons were made using the script: points_to_event_wse_json.py

In [1]:
import os, glob, json
import numpy as np
import pandas as pd
import geopandas

In [2]:
# Get point Ids from points.geojson
qaqc_points = geopandas.read_file('../data/points.geojson')
qaqc_pointIDs = qaqc_points['id'].values

In [3]:
# get the event-based *.json files from ./output
event_wse_json_files = glob.glob('../output/*.json')

In [ ]:
# Create a json file for each point
pt_json_output_dir = '../output/point_jsons'
if not os.path.exists(pt_json_output_dir):
    os.makedirs(pt_json_output_dir)
    
for i, qaqc_point in enumerate(qaqc_pointIDs):
    pt_dict = {}
    for wse_json_file in event_wse_json_files:
        wse_json = json.load(open(wse_json_file, 'r'))
        for pointData in wse_json['pointData']:
            if pointData['qaqc_pointID'] == qaqc_point:
                planName = wse_json['planName']
                pt_dict[planName] = {}
                pt_dict[planName]['datetime'] = wse_json['datetime']
                pt_dict[planName]['wse'] = pointData['wse']
                json.dump(pt_dict, open(f"{pt_json_output_dir}/{qaqc_point}.json", 'w'))

In [ ]:
# Create Data to Plot each point's max WSE as line across events on the x-axis instead of datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go

pt_max_fig = make_subplots(rows=1, cols=1, subplot_titles=[""])
                       
for i, qaqc_point in enumerate(qaqc_pointIDs):
    y_data = []
    x_data = []
    for wse_json_file in event_wse_json_files:
            wse_json = json.load(open(wse_json_file, 'r'))
            for pointData in wse_json['pointData']:
                if pointData['qaqc_pointID'] == qaqc_point:
                    pointData['max_wse'] = max(pointData['wse'])
                    y_data.append(round(pointData['max_wse'],2))
                    x_data.append(wse_json['planName'])
    
    # For each point, add trace to the plot
    pt_max_fig.add_trace(go.Scatter(
        x=x_data, y=y_data,
        hovertext=str(qaqc_point), hoverinfo="text",
        # marker=dict(
        #     color="blue"
        # ),
        showlegend=True, name=str(qaqc_point),
        mode='lines'), row=1, col=1)
        
pt_max_fig.update_layout(height=1500, width=1700, title_text=f"Max WSE vs. Each Event for all points",)

pt_max_fig.write_html('../max_wse.html')

# add in html:
# <div>
#     <img src="./data/basin_points.png" alt="points">
# </div>

# The following cells below are used to create figure plots of each point to its own html file, which wasn't used in the final product.

In [5]:
from PIL import Image
img = Image.open("../data/basin_points.png")
# img

In [6]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

for i, qaqc_point in enumerate(qaqc_pointIDs):
    # make a single point figure
    pt_fig = go.Figure()
    pt_fig = make_subplots(rows=2, cols=1, subplot_titles=["WSE", ""], #vertical_spacing=1, horizontal_spacing=1
                           )
    # open each json file and get the wse for the qaqc_point and add it the subplot for the point.
    for wse_json_file in wse_json_files:
        wse_json = json.load(open(wse_json_file, 'r'))
        for pointData in wse_json['pointData']:
            if pointData['qaqc_pointID'] == qaqc_point:
                # all_pts_fig.add_trace(go.Scatter(x=wse_json['datetime'], y=pointData['wse'], mode='lines', name=wse_json['planName']), row=i+1, col=1)
                pt_fig.add_trace(go.Scatter(
                x=wse_json['datetime'], y=pointData['wse'],
                hovertext=wse_json['planName'], hoverinfo="text",
                # marker=dict(
                #     color="blue"
                # ),
                showlegend=True, name=wse_json['planName'],
                mode='lines'), row=1, col=1)
            # ))
    # Add seond trace thats empty to add the image to the subplot
    pt_fig.add_trace(go.Scatter(
        x=None, y=None, hovertext=None, showlegend=False, mode='lines'), row=2, col=1, )
    pt_fig.update_layout(height=1500, width=1900, title_text=f"WSE for all events at point {qaqc_point}")
    pt_fig.update_layout(xaxis2=dict(showticklabels=False, showgrid=False, zeroline=False, fixedrange=True),
                        yaxis2=dict(showticklabels=False, showgrid=False, zeroline=False),
                        yaxis_domain = [.7,1],
                        yaxis2_domain = [0, .58],
                        xaxis_domain = [0, 1],
                        xaxis2_domain = [0, .445],
                        )

    pt_fig.add_layout_image(
            row=2,
            col=1,
            source=img,
            xref="paper", yref="paper",
            x=-1, y=-1,
            xanchor="left",
            yanchor="bottom",
            sizex=10,
            sizey=5,
            sizing="contain",
            layer="above",

        )

    # export point figure
    pt_fig.update_layout(height=1500, width=1700, title_text=f"WSE for all events at point {qaqc_point}")
    pt_fig.write_html(f'../output/figs/{qaqc_point}.html')

# export the figure to an html file
# all_pts_fig.update_layout(height=2000, width=1000, title_text="WSE for all events at all points")
# all_pts_fig.write_html('all_pts_fig.html')

In [7]:
pt_fig = go.Figure()
pt_fig = make_subplots(rows=2, cols=1, subplot_titles=["WSE", ""], horizontal_spacing=.1, vertical_spacing=.02)
pt_fig.add_trace(go.Scatter(
            x=wse_json['datetime'], y=pointData['wse'],
            hovertext=wse_json['planName'], hoverinfo="text",
            # marker=dict(
            #     color="blue"
            # ),
            showlegend=True, name=wse_json['planName'],
            mode='lines'), row=1, col=1)
        # ))
pt_fig.add_trace(go.Scatter(
    x=None, y=None, hovertext=None, showlegend=False, mode='lines'), row=2, col=1, )
pt_fig.update_layout(height=1500, width=1700, title_text=f"WSE for all events at point {qaqc_point}")
pt_fig.update_layout(xaxis2=dict(showticklabels=False, showgrid=False, zeroline=False, fixedrange=True),
                     yaxis2=dict(showticklabels=False, showgrid=False, zeroline=False),
                     yaxis_domain = [.7,1],
                     yaxis2_domain = [0, .58],
                     xaxis_domain = [0, 1],
                     xaxis2_domain = [0, .445],
                    )

pt_fig.add_layout_image(
        row=2,
        col=1,
        source=img,
        xref="paper", yref="paper",
        x=-1, y=-1,
        xanchor="left",
        yanchor="bottom",
        sizex=10,
        sizey=5,
        sizing="contain",
        layer="above",

    )

pt_fig